The goal of this tutorial is to use Pytorch to train a Deep Q Learning(DQN) agent on the `CartPole-v0` task from the OpenAI Gym.

# Import packages

In [2]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as T

In [4]:
env = gym.make("CartPole-v0").unwrapped

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display
plt.ion()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/Users/qianqu/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


# Replay Memory

Experience replay will help us to handle two things:
* Avoid forgetting previous experiences.
* Reduce correlations between experiences.



We need to classes:

* `Transition` - a tuple representing a single transition in our environment. It maps (state, action) pairs to their (next_state, reward)
* `ReplayMemory` - a cyclic buffer of bounded size that holds the transitions observed recently. It also implement a `.sample()` method for selecting a random batch of transitions for training.

In [5]:
Transition = namedtuple('Transition',
                       ('state', 'action', 'next_state', 'reward'))

In [6]:
class ReplayMemory(object):
    
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0
        
    def push(self, *args):
        """Save a transition"""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        
        self.memory[self.position] = Transition(*args)
        self.position = (self.position+1) % self.capacity
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)
    
    

# Q-network

Our Q-network would be a convolutional neural network that takes in the difference between the current and previous screen patches. 

In [7]:
class DQN(nn.Module):
    def __init__(self, h, w, outputs):
        super(DQN, self).__init__()
        
        self.conv1 = nn.Conv2d(3,16,kernel_size=5,stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16,32,kernel_size=5,stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32,32,kernel_size=5,stride=2)
        self.bn3 = nn.BatchNorm2d(32)
        
        def conv2d_size_out(size, kernel_size=5, stride=2):
            # calculate the output size of convolutional layer
            return (size - kernel_size) // stride + 1
        
        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_size = convw * convh * 32
        
        self.head = nn.Linear(linear_size, outputs)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        out = self.head(x.view(x.size(0),-1)) # x.size(0) is batch size
        return out